In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [2]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [3]:
data = requests.get(standings_url)

In [4]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

In [5]:
team_urls = [f"https://fbref.com{l}" for l in links]

In [6]:
data = requests.get(team_urls[0])

In [7]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

In [8]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [9]:
data = requests.get(f"https://fbref.com{links[0]}")

In [10]:
shooting = pd.read_html(data.text, match="Shooting")[0]

In [11]:
shooting.head()

For Barcelona                                                            \
           Date   Time          Comp        Round  Day Venue Result GF GA   
0    2022-08-13  21:00       La Liga  Matchweek 1  Sat  Home      D  0  0   
1    2022-08-21  22:00       La Liga  Matchweek 2  Sun  Away      W  4  1   
2    2022-08-28  19:30       La Liga  Matchweek 3  Sun  Home      W  4  0   
3    2022-09-03  21:00       La Liga  Matchweek 4  Sat  Away      W  3  0   
4    2022-09-07  21:00  Champions Lg  Group stage  Wed  Home      W  5  1   

                      ... Standard               Expected                    \
            Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh G-xG   
0     Rayo Vallecano  ...     17.0  1.0  0     0      1.9  1.9    0.09 -1.9   
1      Real Sociedad  ...     14.6  0.0  0     0      2.1  2.1    0.14  1.9   
2         Valladolid  ...     14.4  1.0  0     0      2.6  2.6    0.12  1.4   
3            Sevilla  ...     16.0  2.0  0     0      4.0  4.0    0.24 -1.0   
4  cz Viktoria Plzeň  ...     16.2  0.0  0     0      2.3  2.3    0.11  2.7   

          Unnamed: 25_level_0  
  np:G-xG        Match Report  
0    -1.9        Match Report  
1     1.9        Match Report  
2     1.4        Match Report  
3    -1.0        Match Report  
4     2.7        Match Report  

[5 rows x 26 columns]

In [12]:
shooting.columns = shooting.columns.droplevel()

In [13]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [14]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,4-3-3,Alejandro Hernández,Match Report,NaN,21,5,17.0,1.0,0,0
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,3-2-4-1,José Luis Munuera,Match Report,NaN,15,7,14.6,0.0,0,0
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,4-3-3,Ricardo de Burgos,Match Report,NaN,24,9,14.4,1.0,0,0
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,4-3-3,Antonio Matéu Lahoz,Match Report,NaN,18,5,16.0,2.0,0,0
4,2022-09-07,21:00,Champions Lg,Group stage,Wed,Home,W,5,1,cz Viktoria Plzeň,...,4-3-3,Lawrence Visser,Match Report,NaN,20,10,16.2,0.0,0,0


In [15]:
years = list(range(2023, 2020, -1))
all_matches = []

In [16]:
standings_url = "https://fbref.com/en/comps/12/La-Liga-Stats"

In [18]:
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        time.sleep(5)
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "La Liga"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(5)

In [19]:
len(all_matches)

79

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-13,21:00,La Liga,Matchweek 1,Sat,Home,D,0,0,Rayo Vallecano,...,Match Report,NaN,21.0,5.0,17.0,1.0,0.0,0.0,2023,Barcelona
1,2022-08-21,22:00,La Liga,Matchweek 2,Sun,Away,W,4,1,Real Sociedad,...,Match Report,NaN,15.0,7.0,14.6,0.0,0.0,0.0,2023,Barcelona
2,2022-08-28,19:30,La Liga,Matchweek 3,Sun,Home,W,4,0,Valladolid,...,Match Report,NaN,24.0,9.0,14.4,1.0,0.0,0.0,2023,Barcelona
3,2022-09-03,21:00,La Liga,Matchweek 4,Sat,Away,W,3,0,Sevilla,...,Match Report,NaN,18.0,5.0,16.0,2.0,0.0,0.0,2023,Barcelona
5,2022-09-10,18:30,La Liga,Matchweek 5,Sat,Away,W,4,0,Cádiz,...,Match Report,NaN,16.0,8.0,14.9,0.0,0.0,0.0,2023,Barcelona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,2020-07-05,17:00,La Liga,Matchweek 34,Sun,Home,L,0,1,Leganés,...,Match Report,NaN,7.0,2.0,18.9,1.0,0.0,0.0,2021,Espanyol
51,2020-07-08,22:00,La Liga,Matchweek 35,Wed,Away,L,0,1,Barcelona,...,Match Report,NaN,11.0,2.0,18.1,1.0,0.0,0.0,2021,Espanyol
52,2020-07-12,14:00,La Liga,Matchweek 36,Sun,Home,L,0,2,Eibar,...,Match Report,NaN,14.0,3.0,20.8,3.0,0.0,0.0,2021,Espanyol
53,2020-07-16,21:00,La Liga,Matchweek 37,Thu,Away,L,0,1,Valencia,...,Match Report,NaN,18.0,6.0,21.3,3.0,0.0,0.0,2021,Espanyol


In [23]:
match_df.to_csv("spanish_matches.csv")